# Contents <a id='top'></a>

1. <a href=#intro>Introduction</a>
1. <a href=#pre-proc>NLP Pre-processing</a>
1. <a href=#bow>Bag of Words Model</a>
1. <a href=#lda>Topic Modeling</a>
1. <a href=#senti>Sentiment Analysis</a>
1. <a href=#ref>References and Links</a>

<a id='intro'></a>
# 1. Introduction
<a href=#top>(back to top)</a>

In our world, Natural Language Processing (NLP) is used in several scenarios. For example, 

    * phones and handheld computers support predictive text and handwriting recognition; 
    * web search engines give access to information locked up in unstructured text; 
    * machine translation allows us to understand texts written in languages that we do not know; 
    * text analysis enables us to detect sentiment in tweets and blogs.

But as we begin to explore Natural Language, we realise that it is an extremely difficult subject. Here are some specific points to note:

1. Some words mean different things in different contexts, but us humans know which meaning is being used.
    * He **served** the **dish**.
1. In the following two sentences, the word "by" has different meanings:
    * The lost children were found by the lake.
    * The lost children were found by the search party.
1. In the following cases, we (humans) can resolve what "they" is referring to, but it is not easy to generate a simple rule that a computer can follow.
    * The thieves stole the paintings. They were subsequently recovered.
    * The thieves stole the paintings. They were subsequently arrested.
1. How can we get a computer to understand the following tweet?:
    * "Wow. Great job st@rbuck's. Best cup of coffee ever."

## Definitions

Before we go on, it would be useful to establish some terminology.

A *document* is a string of text. It could contain an essay on a particular topic, a paragraph, a single sentence, or even just a tweet of 140 characters. 

A *text corpus* is a large collection of text documents. The plural of corpus is *corpora*. 

Each document is typically split into *tokens* before the actual NLP is done. Consider the sentence

A typical *tokenisation* would be 

How we tokenize and pre-process things will affect our final results. We shall discuss this more in a minute.

## Goals of NLP

Corpora could come in different formats and have different structures.

![text-corpus-structure](../figs/text-corpus-structure_nlp.png)

Sometimes, the format or relationships between the documents in a corpus restrict or direct us toward certain types of analyses. In general, the goals of NLP can be varied. Here are some of the use-cases that we shall discuss:

1. *Computing similarities between new documents and existing ones in a corpus*. This is an unsupervised approach. If the new document can be considered a "query", then this can be used to retrieve and prioritise documents that are relevant to the query.
2. *Identifying "topics" within a corpus*. This is also an unsupervised technique, and allows us to identify the salient topics of a new document automatically. This could be useful in a customer feedback setting, because it would allow quick allocation or prioritisation of resources. This approach requires one to decide on the number of topics. It typically also requires some study of the topics in order to interpret and verify them.
3. *Identifying the overarching sentiment of a document*. Using a lexicon of words and their tagged sentiments, we can assess whether the sentiment in a document is mostly positive or negative.

There are some other applications and ideas that we may not have time to cover:

1. *Named Entity Recognition* If we can identify the particular real-world entities that a query is referring to, we would be able to provide more accurate answers from the corpus. This concept is utilised in chatbots.
2. *Word Embeddings*. This procedure provides a vector representation of a word, in the context of its semantic usage. With this, we can study the context of a word in this corpus, and understand what it means here.

# Wine Reviews Dataset

A dataset containing wine reviews is accessible from [Kaggle](https://www.kaggle.com/zynicide/wine-reviews). We shall work with one of the csv files. It contains 130,000 rows. 

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
wine_reviews = pd.read_csv("../data/winemag-data-130k-v2.csv", index_col=0)

In [ ]:
wine_reviews

In [ ]:
np.where(wine_reviews.duplicated(subset=None, keep='last') == True)

In [ ]:
np.where(wine_reviews.duplicated(subset=None, keep='first') == True)

In [ ]:
wine_reviews[wine_reviews.description == wine_reviews.description[2408]]

In [ ]:
np.sum(wine_reviews.duplicated())

In [ ]:
wine_reviews.drop_duplicates(inplace=True)

In [ ]:
wine_reviews.shape

In [ ]:
wine_reviews.points.hist();

In [ ]:
wine_reviews.description.values[:5]

The "descriptions" column contains the review for a particular wine by a user, whose name and twitter handle are provided. Also included is information such as the price, originating county, region of the wine, and so on. In our activity, we are going to apply NLP techniques to the wine reviews.

<a id='pre-proc'></a>
# 2. NLP Pre-Processing
<a href=#top>(back to top)</a>

Text documents consist of sentences of varying lengths. Usually, the first step to analysing a document is to break it up into pieces. This process is known as **tokenizing**. When tokenizing a document, we can do it at several levels of resolution: at the sentence, line, word or even punctuation level.

For today, we are going to to simply tokenize each document by whitespace. This is easily done using `.split()` within built-in Python. But after that, we need to further pre-process the tokens.

In [ ]:
from gensim.parsing.preprocessing import *
from gensim import models, corpora, similarities
from gensim.models.coherencemodel import CoherenceModel
import pprint

pp = pprint.PrettyPrinter(indent=4, compact=True,)

The `gensim` package includes a module for pre-processing text strings. Here is a list of some of the functions there:

* `strip_multiple_whitespaces`
* `strip_non_alphanum`
* `strip_numeric`
* `strip_punctuation`
* `strip_short`

In [ ]:
r1 = wine_reviews.description.values[0]

In [ ]:
pp.pprint(r1.lower().split())

In [ ]:
pp.pprint(strip_punctuation(r1.lower()).split())

In [ ]:
sent1 = 'This contains a number 144.'

In [ ]:
pp.pprint(strip_numeric(sent1.lower()).split())

Since what we are about to do in the initial part of our activity is based on frequency counts of tokens, apart from some of the above steps, we are also going to remove common "filler" words that could end up skewing the eventual probability distributions of counts. These filler words are known as *stop words*. Linguists came up with them. They vary from model to model, from Python package to package, and of course, from language to language.

It depends on your particular application if stop-word removal is meaningful or not. At times, it is only done in order to speed up the training of a model. However, it is possible to change the entire meaning of a sentence by removing stop-words.

In [ ]:
s1 = 'I do not like eating ice cream.'

In [ ]:
remove_stopwords(s1).split()

In [ ]:
#pp.pprint(STOPWORDS)

For us, we are going to apply this list of filters to each review:

1. strip_punctuation(),
2. strip_multiple_whitespaces(),
3. strip_numeric(),
4. remove_stopwords(),
5. strip_short(),
6. lemmatize

Lemmatizing a word is to reduce it to its root word. You will come across stemming whenever you read about lemmatizing. In both cases, we wish to reduce a word to its root word so that we do not have to deal with multiple variations of a token, such as ate, eating, and eats.

When we stem a word, the prefix and/or suffix will be removed according to a set of rules. Since it is primarily rule-based, the resulting word may not be an actual English word.

Like stemming, lemmatizing also aims to reduce a word to its root form. However, it differs from stemming in that the final word must be a proper English language word. For this purpose, the algorithm has to be supplied with a lexicon or dictionary, along with the text to be lemmatized.

Here is an example that demonstrates the differences.

In [ ]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
porter = PorterStemmer()
wn = WordNetLemmatizer()

demo_sentence = 'Cats and ponies have a meeting'.split()

In [ ]:
demo_sentence

In [ ]:
[porter.stem(x) for x in demo_sentence]

In [ ]:
#import nltk
#nltk.download('omw-1.4')
#nltk.download('wordnet')
[wn.lemmatize(x) for x in demo_sentence]

Now let us go ahead and perform the pre-processing on the wine reviews.

In [ ]:
CUSTOM_FILTER = [lambda x: x.lower(), strip_punctuation, 
                 strip_multiple_whitespaces, strip_numeric, 
                 remove_stopwords, strip_short]

In [ ]:
all_review_strings = wine_reviews.description.values

In [ ]:
all_strings_tokenized = [preprocess_string(x, CUSTOM_FILTER) for x in all_review_strings]

In [ ]:
pp.pprint(all_strings_tokenized[1])

At this point in time, what we have is a list of lists. Each sub-list contains the tokens for a particular wine_review. For instance, the original review for row 234 was:

In [ ]:
pp.pprint(wine_reviews.description.values[233])

In [ ]:
pp.pprint(all_strings_tokenized[233])

`gensim` does not have a lemmatizer, so we use the nltk lemmatizer on each token.

In [ ]:
preprocessed_corpus = [[wn.lemmatize(w) for w in dd ] for dd in all_strings_tokenized]

In [ ]:
pp.pprint(preprocessed_corpus[10])

In [ ]:
pp.pprint(all_strings_tokenized[10])

<a id='bow'></a>
# 3. Tfidf Model (Bag-of-Words)
<a href=#top>(back to top)</a>

In order to perform machine learning on text documents, we have to convert the text content into numerical features. The first method we shall study is to use the Bag of Words (BoW) representation. Here's how it is done, starting from a corpus:

1. Build a vocabulary of words from the corpus and assign a fixed integer id to each word.
2. For each document $i$ in the collection, count the number of occurrences of word $j$ in the dictionary.
3. Store this count in the $(i,j)$-th entry of a matrix.

At this point we have a numerical representation of the corpus. Let us take a look at a simple example instead of the wine review one.

In [ ]:
raw_docs = ["Here are some very simple basic sentences.",
            "So basic that they won’t be very interesting , I’m afraid. ", 
            """The point of these basic examples is to learn how basic text 
            counting works on *very simple* data, so that we are not afraid when 
            it comes to larger text documents. The sentences are here just to provide words."""]

In [ ]:
raw_docs[2]

We apply the default pre-processing filters from gensim and inspect the tokens now.

In [ ]:
example_corpus = preprocess_documents(raw_docs)

In [ ]:
pp.pprint(example_corpus)

In what proceeds, we build a vocabulary of terms in the corpus, and convert them to the Bag of Words representation.

In [ ]:
d1 = corpora.Dictionary(example_corpus)

In [ ]:
d1

In [ ]:
d1[17]

In [ ]:
pp.pprint(list(d1.items()))

In [ ]:
print(d1)

In [ ]:
d1.token2id['basic']

In [ ]:
d1.id2token[16]

In [ ]:
example_bow = [d1.doc2bow(x) for x in example_corpus]
pp.pprint(example_bow)

Now, we have a sparse representation of each document in our corpus. We can in fact convert it back to the processed corpus mode at any time.

The counts indicate the number of times each feature (or words) were present in the document. As you might observe, longer documents tend to contain larger counts (see document 3, which has many more 1's and even a couple of 2's. Thus, instead of dealing with counts, 
we shall convert each row into a vector of length 1. Words that appear in all documents will be weighted down by this transformation, since these do not help to distinguish the document from others. This transformation is known as the TF-IDF transformation.

Instead of the raw counts, we define:

* $N$ to be the number of documents ($N=3$ in the little example above).
* $tf_{i,j}$ to be the frequency of term $i$ in document $j$.
* $df_{i}$ to be the frequency of term $i$ across all documents.
* $w'_{i,j}$ to be: 

\begin{equation}
w'_{i,j} = tf_{i,j} \times \log_2 \left( \frac{N}{df_{i}} \right)
\end{equation}

Then the final $w_{i,j}$ for term $i$ in document $j$ is the normalised version of $w'_{i,j}$ across the terms in that document. 

Consider the word "sentences", in document id 02 (the third document).
* $N = 3$
* $tf_{i,j} = 1$
* $df_{i} = 2$

Thus

\begin{equation}
w'_{ij} = 1 \times \log_2 ( 3/2) = 0.58
\end{equation}

In [ ]:
example_tfidf = models.TfidfModel(example_bow, normalize=False)

In [ ]:
example_bow[0]

In [ ]:
pp.pprint(example_tfidf[example_bow[0]])

In [ ]:
pp.pprint(example_bow[0])

In [ ]:
pp.pprint(example_tfidf[d1.doc2bow(['basic', 'afraid', 'data'])])

Now let us move back to the wine reviews and create the tfidf transform.

In [ ]:
dct = corpora.Dictionary(preprocessed_corpus)

In [ ]:
list(dct.items())[-1]

In [ ]:
dct.dfs[31830]

In [ ]:
np.log2(119988/31325)

In [ ]:
#dct.num_docs
bow_corpus = [dct.doc2bow(text) for text in preprocessed_corpus]

In [ ]:
pp.pprint(bow_corpus[0])

In [ ]:
pp.pprint(preprocessed_corpus[0])

In [ ]:
tfidf = models.TfidfModel(dictionary=dct)

In [ ]:
pp.pprint(tfidf[dct.doc2bow(preprocessed_corpus[0])])
#pp.pprint(tfidf[bow_corpus[0]])

Before we can find matching documents, we build a similarity index, so that matches are returned quicker. We try something simple at first:

> Which documents/reviews are similar to the first one?

In [ ]:
index = similarities.Similarity(None, corpus=tfidf[bow_corpus], num_features=len(dct))

In [ ]:
sims = index[tfidf[bow_corpus[0]]]

In [ ]:
sims

These are the most similar reviews to review id 0. Of course, the first review itself is there!

In [ ]:
np.argsort(-sims)[:10]

In [ ]:
pp.pprint(wine_reviews.description[[0, 108537, 2000, 9929]].values)

In [ ]:
wine_reviews.loc[[0, 108537, 2000, 9929], :]

In [ ]:
#sims = index[tfidf[bow_corpus[20]]]

Now let us try an arbitrary query. Suppose we are interested in "fruity full palate" wines.

In [ ]:
q1 = 'acidic oaky chardonnay'.split()
q1

In [ ]:
sims = index[tfidf[dct.doc2bow(q1)]]

In [ ]:
q1_results = np.argsort(-sims)[:10]
q1_results

In [ ]:
pp.pprint(wine_reviews.description.values[q1_results])

In [ ]:
wine_reviews.iloc[q1_results, :]

<a id='lda'></a>
# 4. Topic Modeling
<a href=#top>(back to top)</a>

The LDA (Latent Dirichlet Allocation) model assumes the following intuitive generative process for the documents:

1. There is a set of $K$ topics that the documents come from. Each document contains words from several topics. There is a probability mass function on the topics for each document.
2. For each topic, there is a probability mass function for the distribution of words in that topic. 

At the end of LDA topic modeling, we will be able to tell, for a particular (new or old) document: the weight combination of the topics for that document. For each topic, we would be able to tell the terms that are salient.

LDA only gives us the probabilistic weights &mdash; we have to interpret them ourselves.

In [ ]:
lda1 = models.LdaModel(corpus=bow_corpus, num_topics=20, id2word=dct)

In [ ]:
pp.pprint(lda1.get_topic_terms(19))

In [ ]:
pp.pprint([dct.id2token[x] for x in [290, 581, 167, 851, 131, 100, 1041, 473, 115, 54]])

In [ ]:
pp.pprint(lda1.show_topics(num_topics = 20, num_words=10))

In [ ]:
pp.pprint(lda1.get_document_topics(bow_corpus[0]))

In [ ]:
pp.pprint(preprocessed_corpus[0])

In [ ]:
# input is topic distribution for a particular document, e.g.
# output of lda1.get_document_topics(bow_corpus[0])
#
# output is True if that document has more than threshold probability for topic_num
def check_for_topic(topic_pmf, topic_num, threshold):
    for topic, prob in topic_pmf:
        if (topic == topic_num) & (prob > threshold):
            return True
    return False

In [ ]:
tf_list = [check_for_topic(lda1.get_document_topics(x), 3, 0.8) for x in bow_corpus]

In [ ]:
wine_reviews.description.values[np.where(tf_list)[0][:10]]

In [ ]:
pp.pprint(lda1.get_document_topics(bow_corpus[8]))

In [ ]:
lda1.get_topic_terms(13, 8)

In [ ]:
[dct.id2token[x] for x in [16, 2, 58, 54, 38, 41, 254, 0]]

In [ ]:
pp.pprint([dct.id2token[x] for x in [125, 114, 10, 3, 0, 2, 16, 133]])

In [ ]:
lda1.log_perplexity(bow_corpus)

In [ ]:
coh = CoherenceModel(model=lda1, corpus=bow_corpus, coherence='u_mass')

coh.get_coherence_per_topic()

In [ ]:
lda1.print_topic(4)

In [ ]:
lda1.print_topic(0)

In [ ]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()

In [ ]:
vis = pyLDAvis.gensim_models.prepare(lda1, bow_corpus, dictionary=lda1.id2word)

In [ ]:
#vis
pyLDAvis.save_html(vis, 'vis.html')

<a id='senti'></a>
# 5. Sentiment Analysis
<a href=#top>(back to top)</a>


In [ ]:
from nltk.sentiment.vader import SentiText, SentimentIntensityAnalyzer

In [ ]:
#import nltk
#nltk.download('vader_lexicon')

In [ ]:
ss = SentimentIntensityAnalyzer()

In [ ]:
ss.polarity_scores('I really hate this class!! :-(')

In [ ]:
ss.polarity_scores('I really hate this class!! :-D')

In [ ]:
wine_reviews['sentiment'] = wine_reviews.description.apply(lambda x: ss.polarity_scores(x)['compound'])

In [ ]:
wine_reviews.sentiment.describe()

In [ ]:
wine_reviews.sentiment.hist();

In [ ]:
pp.pprint(wine_reviews[wine_reviews.sentiment < -0.5].sample(10).description.values)

<a id='ref'></a>
# 6. References
<a href=#top>(back to top)</a>

1. [nltk](https://www.nltk.org/) This site contains the documentation for the nltk and a book on NLP. The book is a little on the old side (no deep-learning based techniques) but still very useful.
2. [gensim documentation](https://radimrehurek.com/gensim/auto_examples/index.html)
3. There are a few DataCamp courses on NLP that you can continue with when you have time. Here are their titles:
    * Introduction to Natural Language Processing in Python.
    * Advanced NLP with spaCy